In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import geevis
from geevis import lulc

In [3]:
import ee
import numpy as np
import pandas as pd

In [4]:
ee.Initialize()

## NLCD

In [149]:
vegas = ee.Geometry.Polygon([[[-115.01184401606046, 36.24170785506492],
          [-114.98849806879484, 36.29928186470082],
          [-115.25628981684171, 36.35238941394592],
          [-115.34692702387296, 36.310348922031565],
          [-115.37988600824796, 36.160811202271944],
          [-115.30298171137296, 36.03653336474891],
          [-115.25628981684171, 36.05207884201088],
          [-115.26590285395109, 36.226199908103695],
          [-115.19174513910734, 36.25499793268206]]])

start = ee.Image(f"USGS/NLCD/NLCD2001")
end = ee.Image(f"USGS/NLCD/NLCD2016")

dataset = geevis.datasets.nlcd2016

title = "Las Vegas Urban Sprawl, 2001 - 2016"

data = geevis.lulc.sample_change(start, end, vegas, dataset=dataset, n=100, scale=30)
plot = geevis.lulc.plot_sankey(data, "2001", "2016", dataset=dataset, max_classes=4, title=title)
plot

## MODIS

In [137]:
glaciers = ee.Geometry.Polygon([[[-130.07633697303308, 55.018689630077546],
          [-130.07633697303308, 54.340663930662444],
          [-128.81565582068933, 54.340663930662444],
          [-128.81565582068933, 55.018689630077546]]])


start = ee.Image("MODIS/006/MCD12Q1/2001_01_01")
end = ee.Image("MODIS/006/MCD12Q1/2019_01_01")

dataset = geevis.datasets.modis_LC_Type1

title = "Glacial Retreat in British Columbia, 2001 - 2019"
data = geevis.lulc.sample_change(start, end, glaciers, dataset=dataset, n=100, scale=500)
# Exclude conifer forest which experienced very little change
plot = geevis.lulc.plot_sankey(data, "2001", "2019", dataset=dataset, max_classes=3, exclude=[1], title=title)
plot

## Custom data

### Classifying plant health

In [148]:
# Load fire perimeters from MTBS data
fires = ee.FeatureCollection("users/aazuspan/fires/mtbs_1984_2018")
# Select the 2014 Happy Camp Complex fire perimeter in California
fire = fires.filterMetadata("Fire_ID", "equals", "CA4179612337420140814")

# Load imagery 1 year after fire and 5 years after fire
immediate = ee.Image("LANDSAT/LC08/C01/T1_TOA/LC08_045031_20150718");
recovery = ee.Image("LANDSAT/LC08/C01/T1_TOA/LC08_046031_20200807");

# Calculate NDVI
immediate_NDVI = post.normalizedDifference(["B5", "B4"]);
recovery_NDVI = pre.normalizedDifference(["B5", "B4"]);

# Reclassify continuous NDVI values into classes of plant health
immediate_class = ee.Image(1) \
  .where(immediate_NDVI.lt(0.3), 0) \
  .where(immediate_NDVI.gt(0.5), 2) \
  .rename("health")

recovery_class = ee.Image(1) \
  .where(recovery_NDVI.lt(0.3), 0) \
  .where(recovery_NDVI.gt(0.5), 2) \
  .rename("health")

### Plotting health data

In [147]:
# Define the labels and colors associated with each class value defined above
labels = {
    0: "Unhealthy",
    1: "Moderate",
    2: "Healthy"
}
palette = {
    0: "#e5f5f9",
    1: "#99d8c9",
    2: "#2ca25f"
}

title="5-Year Postfire Recovery"

data = geevis.lulc.sample_change(immediate_class, recovery_class, fire, band="health", n=100, scale=20)
plot = geevis.lulc.plot_sankey(data, start_label="Prefire", end_label="Postfire", class_labels=labels, class_palette=palette, title=title)

plot